In [1]:
# dev of code to convert a Conv1d layer to a FGN_Conv1D layer

In [2]:
import torch
import torch.nn as nn

In [3]:
import sys
sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

In [4]:
# define layers
in_channels = 2
out_channels = 3
kernel_size = 2
stride = 1

conv1d =  nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)
fgn_conv1d = fgnl.FGN_Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)

In [5]:
conv1d.state_dict()

OrderedDict([('weight',
              tensor([[[ 0.3778,  0.4663],
                       [ 0.0803,  0.0208]],
              
                      [[ 0.3700,  0.0154],
                       [ 0.0856,  0.4929]],
              
                      [[-0.1171,  0.2407],
                       [ 0.1383, -0.3520]]])),
             ('bias', tensor([ 0.4130,  0.3831, -0.3741]))])

In [6]:
fgn_conv1d.state_dict()

OrderedDict([('centers',
              tensor([[[ 0.1003, -0.0468],
                       [ 0.0339, -0.0394],
                       [ 0.1164, -0.0606]],
              
                      [[ 0.2659,  0.0077],
                       [-0.1787,  0.0214],
                       [ 0.0593, -0.0697]]])),
             ('inv_covars',
              tensor([[1.7493, 1.8115],
                      [1.2066, 1.5190],
                      [1.2027, 1.7948]])),
             ('Conv1d.weight',
              tensor([[[ 0.3472, -0.1723],
                       [ 0.0490,  0.3529]],
              
                      [[ 0.4220,  0.3231],
                       [-0.1072,  0.1632]],
              
                      [[ 0.4824, -0.2758],
                       [ 0.4238, -0.3601]]])),
             ('Conv1d.bias', tensor([-0.1370,  0.2160, -0.0739]))])

In [7]:
# convert
fgn_conv1d.Conv1d.weight = conv1d.weight
fgn_conv1d.Conv1d.bias = conv1d.bias
fgn_conv1d.centers = torch.nn.Parameter( 
    torch.stack([(-b/torch.dot(w.flatten(),w.flatten()))*w.flatten()
                 for (b,w) in zip(conv1d.bias,
                                  conv1d.weight)]).reshape(in_channels, out_channels, kernel_size))

In [8]:
# the problem is new_centers has wrong shape

In [9]:
torch.stack([(-b/torch.dot(w.flatten(),w.flatten()))*w.flatten()
                 for (b,w) in zip(conv1d.bias,
                                  conv1d.weight)]).reshape(in_channels, out_channels, kernel_size)

tensor([[[-0.4251, -0.5247],
         [-0.0903, -0.0234],
         [-0.3659, -0.0152]],

        [[-0.0846, -0.4875],
         [-0.2041,  0.4194],
         [ 0.2410, -0.6133]]], grad_fn=<ViewBackward>)

In [10]:
conv1d.state_dict()

OrderedDict([('weight',
              tensor([[[ 0.3778,  0.4663],
                       [ 0.0803,  0.0208]],
              
                      [[ 0.3700,  0.0154],
                       [ 0.0856,  0.4929]],
              
                      [[-0.1171,  0.2407],
                       [ 0.1383, -0.3520]]])),
             ('bias', tensor([ 0.4130,  0.3831, -0.3741]))])

In [11]:
fgn_conv1d.state_dict()

OrderedDict([('centers',
              tensor([[[-0.4251, -0.5247],
                       [-0.0903, -0.0234],
                       [-0.3659, -0.0152]],
              
                      [[-0.0846, -0.4875],
                       [-0.2041,  0.4194],
                       [ 0.2410, -0.6133]]])),
             ('inv_covars',
              tensor([[1.7493, 1.8115],
                      [1.2066, 1.5190],
                      [1.2027, 1.7948]])),
             ('Conv1d.weight',
              tensor([[[ 0.3778,  0.4663],
                       [ 0.0803,  0.0208]],
              
                      [[ 0.3700,  0.0154],
                       [ 0.0856,  0.4929]],
              
                      [[-0.1171,  0.2407],
                       [ 0.1383, -0.3520]]])),
             ('Conv1d.bias', tensor([ 0.4130,  0.3831, -0.3741]))])

In [12]:
# dev code

In [13]:
w = conv1d.weight[0].flatten()
w

tensor([0.3778, 0.4663, 0.0803, 0.0208], grad_fn=<ViewBackward>)

In [14]:
b = conv1d.bias[0]
b

tensor(0.4130, grad_fn=<SelectBackward>)

In [15]:
(-b/torch.dot(w,w))*w

tensor([-0.4251, -0.5247, -0.0903, -0.0234], grad_fn=<MulBackward0>)

In [16]:
(conv1d.weight.shape, conv1d.bias.shape, fgn_conv1d.centers.shape)

(torch.Size([3, 2, 2]), torch.Size([3]), torch.Size([2, 3, 2]))

In [17]:
new_centers = torch.stack([(-b/torch.dot(w.flatten(),w.flatten()))*w.flatten()
               for (b,w) in zip(conv1d.bias, 
                                conv1d.weight)])
print(new_centers)

tensor([[-0.4251, -0.5247, -0.0903, -0.0234],
        [-0.3659, -0.0152, -0.0846, -0.4875],
        [-0.2041,  0.4194,  0.2410, -0.6133]], grad_fn=<StackBackward>)


In [18]:
# test same behavior
inputs = torch.rand((100,in_channels,8000))
print(inputs.shape)

torch.Size([100, 2, 8000])


In [19]:
# increase inv_covar size
fgn_conv1d.inv_covars = torch.nn.Parameter(fgn_conv1d.inv_covars/25000)

In [20]:
fgn_conv1d.inv_covars

Parameter containing:
tensor([[6.9972e-05, 7.2461e-05],
        [4.8263e-05, 6.0758e-05],
        [4.8108e-05, 7.1791e-05]], requires_grad=True)

In [21]:
r1 = conv1d(inputs)
r1.shape

torch.Size([100, 3, 7999])

In [22]:
r2 = fgn_conv1d(inputs)[0]
r2.shape

torch.Size([100, 3, 7999])

In [23]:
sum(abs(r2-r1).flatten())

tensor(0.0017, grad_fn=<AddBackward0>)

In [24]:
all((r2==r1).flatten())

False

In [25]:
# define converter function
def convert_layer_conv1D_to_fgn(classic_layer, fgn_layer,  init_factor=25000):
    # changes the weights of the fgn_layer to match the behavior of the classic_layer
    # the two layers MUST have been created with the same parameters
    
    # some size checks
    assert(fgn_layer.Conv1d.weight.shape==classic_layer.weight.shape)
    assert(fgn_layer.Conv1d.bias.shape==classic_layer.bias.shape)

    # convert params
    fgn_layer.Conv1d.weight = classic_layer.weight
    fgn_layer.Conv1d.bias = classic_layer.bias
    fgn_layer.centers = torch.nn.Parameter(torch.stack([(-b/torch.dot(w.flatten(),w.flatten()))*w.flatten()
                                                        for (b,w) in zip(classic_layer.bias,
                                                                         classic_layer.weight)]).reshape(fgn_layer.centers.shape)
                                          )
    
    # ensure large sigma, enough to mimic behavior of classic_layer
    fgn_layer.inv_covars = torch.nn.Parameter(fgn_layer.inv_covars/init_factor)
    
    # returns nothing

In [26]:
# define layers
in_channels = 3
out_channels = 4
kernel_size = 5
stride = 6

conv1d =  nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)
fgn_conv1d = fgnl.FGN_Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)

inputs = torch.rand((50,in_channels,4000))

In [27]:
conv1d.state_dict()

OrderedDict([('weight',
              tensor([[[-2.0033e-01, -9.5478e-02,  2.5541e-01, -7.5028e-02, -2.1559e-01],
                       [-1.8583e-01,  5.1569e-02,  2.5164e-01,  1.0181e-01, -2.5646e-01],
                       [-1.5950e-01, -1.7417e-01,  2.0022e-01,  1.9771e-01, -1.6366e-01]],
              
                      [[ 2.4948e-01,  2.2273e-01, -2.2336e-01,  1.0936e-02, -2.0587e-01],
                       [-8.7873e-02,  1.2429e-04, -1.5785e-01,  2.4988e-01,  8.1369e-02],
                       [ 1.7202e-01, -2.1298e-02, -3.6783e-02, -1.5823e-01, -9.1511e-02]],
              
                      [[ 2.1292e-01,  1.7634e-01, -8.0644e-02, -1.4680e-01,  2.3455e-02],
                       [ 1.2856e-01,  2.2521e-01,  1.9661e-01, -8.9145e-02, -1.0592e-01],
                       [-4.1008e-02, -1.7740e-01, -9.3840e-02,  1.9926e-01, -5.8858e-02]],
              
                      [[-9.1639e-02, -1.9358e-01,  4.3106e-02,  4.4430e-02, -4.8923e-03],
                       [-1.8

In [28]:
fgn_conv1d.state_dict()

OrderedDict([('centers',
              tensor([[[ 0.2271,  0.0702,  0.0757, -0.1630, -0.0433],
                       [ 0.0295, -0.2036,  0.1519,  0.0337,  0.2011],
                       [-0.0542, -0.0904, -0.0628, -0.0253, -0.0504],
                       [ 0.1269,  0.0906,  0.0871, -0.0823,  0.0221]],
              
                      [[-0.0619, -0.0090,  0.0944,  0.0555,  0.0778],
                       [ 0.0320, -0.1525,  0.0481, -0.1035,  0.0676],
                       [-0.0683, -0.0337,  0.0485,  0.0871,  0.3085],
                       [-0.0954, -0.1151,  0.0520,  0.0801,  0.1007]],
              
                      [[-0.1525, -0.0468, -0.0406, -0.0391,  0.0298],
                       [ 0.0204, -0.0787,  0.0760,  0.1502,  0.0162],
                       [ 0.0154,  0.0226,  0.0367,  0.0433, -0.1870],
                       [ 0.0861,  0.0564, -0.0315,  0.1019, -0.0572]]])),
             ('inv_covars',
              tensor([[0.5504, 0.6020, 0.6304, 0.6030, 0.6366],
       

In [29]:
convert_layer_conv1D_to_fgn(conv1d, fgn_conv1d)

In [30]:
fgn_conv1d.state_dict()

OrderedDict([('centers',
              tensor([[[ 6.8346e-02,  3.2573e-02, -8.7135e-02,  2.5597e-02,  7.3551e-02],
                       [ 6.3398e-02, -1.7593e-02, -8.5850e-02, -3.4735e-02,  8.7493e-02],
                       [ 5.4416e-02,  5.9419e-02, -6.8306e-02, -6.7450e-02,  5.5836e-02],
                       [-4.3287e-02, -3.8647e-02,  3.8754e-02, -1.8974e-03,  3.5721e-02]],
              
                      [[ 1.5247e-02, -2.1565e-05,  2.7389e-02, -4.3356e-02, -1.4118e-02],
                       [-2.9847e-02,  3.6955e-03,  6.3822e-03,  2.7454e-02,  1.5878e-02],
                       [-8.7180e-02, -7.2203e-02,  3.3020e-02,  6.0108e-02, -9.6039e-03],
                       [-5.2638e-02, -9.2215e-02, -8.0503e-02,  3.6501e-02,  4.3371e-02]],
              
                      [[ 1.6791e-02,  7.2639e-02,  3.8423e-02, -8.1588e-02,  2.4100e-02],
                       [-1.2061e-02, -2.5478e-02,  5.6735e-03,  5.8478e-03, -6.4392e-04],
                       [-2.4450e-02, -3.234

In [31]:
r1 = conv1d(inputs)
r1.shape

torch.Size([50, 4, 666])

In [32]:
r2 = fgn_conv1d(inputs)[0]
r2.shape

torch.Size([50, 4, 666])

In [33]:
sum(abs(r2-r1).flatten())

tensor(0., grad_fn=<AddBackward0>)

In [34]:
all((r2==r1).flatten())

True